Escalado de características <br>
Normalizar o estandarizar son técnicas muy similares que cambian el rango de valores que tiene una característica. De este modo, los modelos aprenden más rápido y con mayor solidez.

Ambos procesos se denominan comúnmente escalado de características.

En este ejercicio utilizaremos un conjunto de datos de adiestramiento de perros para predecir cuántos rescates realizará un perro en un año determinado, basándonos en la edad que tenían cuando comenzó su adiestramiento.

Entrenaremos modelos con y sin escalado de características y compararemos su comportamiento y resultados.

Pero primero, vamos a cargar nuestro conjunto de datos e inspeccionarlo:

In [ ]:
import pandas
!wget https://raw.githubusercontent.com/MicrosoftDocs/mslearn-introduction-to-machine-learning/main/graphing.py
!wget https://raw.githubusercontent.com/MicrosoftDocs/mslearn-introduction-to-machine-learning/main/Data/dog-training.csv
!wget https://raw.githubusercontent.com/MicrosoftDocs/mslearn-introduction-to-machine-learning/main/m1b_gradient_descent.py
data = pandas.read_csv("dog-training.csv", delimiter="\t")
data.head()

El conjunto de datos anterior nos dice a qué edad empezó a entrenar un perro, cuántos rescates, de media, ha realizado al año, y otras estadísticas, como qué edad tenía el año pasado, su peso y cuántos rescates realizó en ese periodo.

Obsérvese que también tenemos variables expresadas en distintas unidades, como mes_viejo_cuando_entrenó en meses, edad_último_año en años y peso_último_año en kilogramos.

Tener características en rangos y unidades muy diferentes es un buen indicador de que un modelo puede beneficiarse del escalado de características.

En primer lugar, vamos a entrenar nuestro modelo utilizando el conjunto de datos "tal cual":

In [ ]:
from m1b_gradient_descent import gradient_descent
import numpy
import graphing

# Train model using Gradient Descent
# This method uses custom code that will print out progress as training advances.
# You don't need to inspect how this works for these exercises, but if you are
# curious, you can find it in out GitHub repository
model = gradient_descent(data.month_old_when_trained, data.mean_rescues_per_year, learning_rate=5E-4, number_of_iterations=8000)

Análisis del entrenamiento<br>
Como puede ver en la salida anterior, estamos imprimiendo una estimación de los pesos y el coste calculado en cada iteración.

La última línea de la salida muestra que el modelo dejó de entrenarse porque alcanzó el número máximo de iteraciones permitido, pero el coste podría ser menor si lo hubiéramos dejado correr más tiempo.

Tracemos el modelo al final de este entrenamiento:

In [ ]:
# Plot the data and trendline after training
graphing.scatter_2D(data, "month_old_when_trained", "mean_rescues_per_year", trendline=model.predict)

El gráfico anterior nos dice que cuanto más joven empiece a adiestrarse un perro, más rescates realizará en un año.

Obsérvese que no se ajusta muy bien a los datos (la mayoría de los puntos están por encima de la línea). Esto se debe a que el adiestramiento se interrumpió pronto, antes de que el modelo pudiera encontrar los pesos óptimos.

Normalización de los datos<br>
Utilicemos la estandarización como forma de escalado de características para este modelo, aplicándola a la característica month_old_when_trained:

In [ ]:
# Add the standardized verions of "age_when_trained" to the dataset.
# Notice that it "centers" the mean age around 0
data["standardized_age_when_trained"] = (data.month_old_when_trained - numpy.mean(data.month_old_when_trained)) / (numpy.std(data.month_old_when_trained))

# Print a sample of the new dataset
data[:5]

Observe que los valores de la columna edad_estandarizada_cuando_se_entrenó se distribuyen en un rango mucho más pequeño (entre -2 y 2) y tienen su media centrada en 0.

Visualización de características escaladas<br>
Utilicemos un diagrama de cajas para comparar los valores originales de las características con sus versiones estandarizadas:

In [ ]:
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import plotly.express as px

fig = px.box(data,y=["month_old_when_trained", "standardized_age_when_trained"])
fig.show()

Ahora compare las dos características pasando el ratón por encima del gráfico. Verá que

month_old_when_trained oscila entre 1 y 71 y tiene su mediana centrada en 35.

edad_estandarizada_cuando_se_entrenó oscila entre -1,6381 y 1,6798, y está centrada exactamente en 0.

Entrenamiento con características estandarizadas
Ahora podemos volver a entrenar nuestro modelo utilizando las características estandarizadas de nuestro conjunto de datos:

In [ ]:
# Let's retrain our model, this time using the standardized feature
model_norm = gradient_descent(data.standardized_age_when_trained, data.mean_rescues_per_year, learning_rate=5E-4, number_of_iterations=8000)

Veamos de nuevo el resultado.

A pesar de que todavía se le permite un máximo de 8000 iteraciones, el modelo se detuvo en la marca 5700.

¿Por qué? Porque esta vez, utilizando la función estandarizada, pudo alcanzar rápidamente un punto en el que ya no se podía mejorar el coste.

En otras palabras, "convergió" mucho más rápido que la versión anterior.

Trazado del modelo normalizado
Ahora podemos trazar el nuevo modelo y ver los resultados de la normalización:

In [ ]:
# Plot the data and trendline again, after training with standardized feature
graphing.scatter_2D(data, "standardized_age_when_trained", "mean_rescues_per_year", trendline=model_norm.predict)

Parece que este modelo se ajusta a los datos mucho mejor que el primero.

El modelo estandarizado muestra una mayor pendiente y los datos están ahora centrados en 0 en el eje X, factores ambos que deberían permitir que el modelo convergiera más rápidamente.

¿Pero cuánto más rápido?

Hagamos una comparación entre modelos para visualizar las mejoras.

In [ ]:
cost1 = model.cost_history
cost2 = model_norm.cost_history

# Creates dataframes with the cost history for each model
df1 = pandas.DataFrame({"cost": cost1, "Model":"No feature scaling"})
df1["number of iterations"] = df1.index + 1
df2 = pandas.DataFrame({"cost": cost2, "Model":"With feature scaling"})
df2["number of iterations"] = df2.index + 1

# Concatenate dataframes into a single one that we can use in our plot
df = pandas.concat([df1, df2])

# Plot cost history for both models
fig = graphing.scatter_2D(df, label_x="number of iterations", label_y="cost", title="Training Cost vs Iterations", label_colour="Model")
fig.update_traces(mode='lines')
fig.show()

Este gráfico muestra claramente que el uso de un conjunto de datos estandarizados permitió que nuestro modelo convergiera mucho más rápido. Alcanzar el coste más bajo y encontrar los pesos óptimos requirió un número mucho menor de iteraciones.

Esto es muy importante cuando se está desarrollando un nuevo modelo, ya que permite iterar más rápido, pero también cuando el modelo se despliega en un entorno de producción, ya que requerirá menos tiempo de cálculo para el entrenamiento y costará menos que un modelo "lento".

Resumen<br>
En este ejercicio hemos tratado los siguientes conceptos:<br>

Las técnicas de scalaing de características se utilizan para mejorar la eficiencia de los modelos de entrenamiento<br>
Cómo añadir una característica normalizada a un conjunto de datos<br>
Cómo visualizar características estandarizadas y compararlas con sus valores originales<br>
Por último, hemos comparado el rendimiento de los modelos antes y después de utilizar características estandarizadas, utilizando gráficos para visualizar las mejoras.